### Example 17 - Evaluating Relisting Targets

In this example, we'll attempt to find good targets for a relisting strategy as described in the text.  We'll evaluate relisting targets according to the following criteria:

1. **High Buy Volume** - There should be ample buy volume in the market.  Our main risk with this strategy is that we are unable to sell our orders before enough competition arrives to undercut us below profitability.  As long as buy volume is strong, this risk is low.
2. **Low Sell Volume** - There should *not* be too much sell volume in the market.  A market with both strong buy and sell volume will make it too easy for competitors to buy and sell competitively.  In this type of market, relisting only helps our competitors by pushing up ask prices even further.  We'll have competition and it will be difficult to sell.
3. **Low Sell Side Competition** - There shouldn't be too much competition on the sell side.  The more competition, the more we'll need to push up sell prices to be profitable.
4. **Cost** - It should not be too expensive to buy out the ask side of the market.  Our strategy will need to set a limit for what we are willing to pay.  Note that we may not need to buy out the entire ask side, just enough to allow the placement of sell orders at profit.

Once we've selected likely relisting targets, we'll consider the likely success of a strategy like this.

In [1]:
# Standard imports
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
# EveKit imports`
from evekit.reference import Client
from evekit.util import convert_raw_time

In [2]:
# As in previous examples, we'll consider opportunities in The Forge at the busiest station in Jita.
# We'll look for market making targets which are consistently strong over an approximately five
# month historical period.  It's likely such assets will continue to be good relisting targets
# in the near future but, of course, historical performance does not guarantee future returns.
#
sde_client = Client.SDE.get()
region_query = "{values: ['The Forge']}"
station_query = "{values: ['Jita IV - Moon 4 - Caldari Navy Assembly Plant']}"
region_id = sde_client.Map.getRegions(regionName=region_query).result()[0][0]['regionID']
station_id = sde_client.Station.getStations(stationName=station_query).result()[0][0]['stationID']
date_range = pd.date_range(datetime.datetime(2017, 1, 7), datetime.datetime(2017, 5, 20))
print("Using region_id=%d, station_id=%d from %s to %s" % (region_id, station_id, str(date_range[0]), str(date_range[-1])))

Using region_id=10000002, station_id=60003760 from 2017-01-07 00:00:00 to 2017-05-20 00:00:00


In [3]:
# We'll filter for general liquidity first, starting from all available types.  This will eliminate
# assets with obvious flaws.
#
market_types = Client.SDE.load_complete(sde_client.Inventory.getTypes, marketGroupID="{start: 0, end: 1000000000}")
market_type_map = {}
for x in market_types:
    market_type_map[x['typeID']] = x
len(market_types)

11782

In [4]:
# As we've done with other market making targets, for this example we'll limit ourselves to 
# Saturdays only.  Thus we'll only load market history for this restricted date range.
#
from evekit.marketdata import MarketHistory
sat_date_range = [x for x in date_range if x.weekday() == 5]
market_history = MarketHistory.get_data_frame(dates=sat_date_range, types=market_type_map.keys(), regions=[region_id], 
                                              config=dict(local_storage=".", tree=True, skip_missing=True, verbose=True))

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [5]:
# Include the liquidity filter framework from previous examples.
#
def liquid_types(history, liquidp, verbose=False):
    # Result is a map from region to set of liquid types for that region
    # Iterate through all types contained in the history object
    liquid_map = {}
    count = 0
    # Iterate through all regions and types
    for next_region in history.region_id.unique():
        liquid_set = set()
        by_region = history[history.region_id == next_region]
        for next_type in by_region.type_id.unique():
            by_type = by_region[by_region.type_id == next_type]
            if liquidp(next_region, next_type, by_type):
                liquid_set.add(next_type)
            count += 1
            if count % 1000 == 0 and verbose:
                print("Tested %d (region, type) pairs" % count)
        liquid_map[next_region] = liquid_set
    return liquid_map

In [6]:
# In this example, we'll filter for assets which trade every day in our historic range, 
# with a reasonable number of orders.  Unlike the pure market making strategy, we're ignoring
# the price averaged volume since we'll be adding more detailed checks later to determine
# exact trade volume.  If it turns out that good candidates always meet a certain price
# averaged volume test, then we can add this criteria back in.
#
def liquidity_filter(min_days, min_count):
    def liquidp(region_id, type_id, history):
        return len(history) >= min_days and \
               len(history[history.order_count < min_count]) == 0
    return liquidp

In [7]:
# We'll use the following values to parameterize the liquidity filter.
#
# Minimum number of orders per day
min_count = 250
# Each type we consider must trade every day
min_values = len(market_history.index.unique())

In [8]:
# Now compute liquid types.  The liquidity filter returns a map from region to the
# set of liquid types in that region.
#
liquid_type_map = liquid_types(market_history, liquidity_filter(min_values, min_count))
len(liquid_type_map[region_id])

218

In [9]:
# Now that we have a reduced set of reasonably liquid types, we'll move to our first two filters
# which compare buy side and sell side volume.  This means we'll need to extract trades from order books
# which means we'll need our trade extraction code from previous examples.  That code starts by 
# calculating a volume threshold which helps us distinguish large trades from cancel orders.  Since
# that calculation uses a window'd average, we need to load a bit more market history which we
# do here.
#
target_types = liquid_type_map[region_id]
ext_date_range = [datetime.datetime(2016, 12, 10), datetime.datetime(2016, 12, 17),
                  datetime.datetime(2016, 12, 24), datetime.datetime(2016, 12, 31)] + sat_date_range
ext_market_history = MarketHistory.get_data_frame(dates=ext_date_range, types=target_types, regions=[region_id], 
                                                  config=dict(local_storage=".", tree=True, skip_missing=True, verbose=True))

Retrieving 2016-12-10 00:00:00...done
Retrieving 2016-12-17 00:00:00...done
Retrieving 2016-12-24 00:00:00...done
Retrieving 2016-12-31 00:00:00...done
Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [10]:
# Now we can compute the thresholds we need for trade inferrence
#
volume_thresh_multiplier = 0.04
volume_thresh_map = {}
for next_type in ext_market_history.groupby(ext_market_history.type_id):
    group_id = next_type[0]
    group_df = next_type[1]
    volume_thresh_map[group_id] = group_df.volume.rolling(window=5, center=False).mean() * volume_thresh_multiplier

In [11]:
# This is our side volume computer from an earlier example.  It will extract trades and compute the total
# buy and sell side trade volume for a given type and day.
#
def compute_side_volume(type_id, dt, order_book, volume_threshold_map):
    buy_volume = 0
    sell_volume = 0
    by_type = order_book[order_book.type_id == type_id]
    vol_limit = volume_threshold_map[type_id][dt]
    #
    # Iterate over consecutive book snapshots looking for order book changes.
    #
    snap_list = list(by_type.groupby(by_type.index))
    snap_pairs = zip(snap_list, snap_list[1:])
    for current, next in snap_pairs:
        current_snap = current[1]
        current_time = current[0]
        next_snap = next[1]
        next_time = next[0]
        # Look for volume changes.  These are trades.
        merged = pd.merge(current_snap, next_snap, on="order_id")
        changed_orders = merged[merged.volume_x != merged.volume_y]
        for next_change in changed_orders.index:
            # Create the trade object
            next_line = changed_orders.ix[next_change]
            amount = next_line.volume_x - next_line.volume_y 
            if next_line.buy_x:
                buy_volume += amount
            else:
                sell_volume += amount
        # Look for removed orders.  These are either a fully filled order or a cancel.
        removed_orders = set(current_snap.order_id).difference(set(next_snap.order_id))
        current_order_list = list(current_snap.order_id)
        for order_id in removed_orders:
            order_count = current_order_list.count(order_id)
            next_line = current_snap[current_snap.order_id == order_id].ix[current_time]
            # If the volume of a removed order does not exceed the threshold, then it's a trade
            if order_count > 1:
                # Handle duplicate orders found in some data
                next_line = next_line.iloc[0]
            if next_line.volume <= vol_limit:
                if next_line.buy:
                    buy_volume += next_line.volume
                else:
                    sell_volume += next_line.volume
    # Return result
    return (buy_volume, sell_volume)

In [12]:
# This code iterates through each order book in our date range and computes the buy and sell
# volume for each type on each day.  We'll assemble these values into a DataFrame for
# further analysis.
#
from evekit.marketdata import OrderBook

side_volume_data = []
for next_date in sat_date_range:
    order_book = OrderBook.get_data_frame(dates=[next_date], types=target_types, regions=[region_id], 
                                          config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                      fill_gaps=True, verbose=True))
    order_book = order_book[order_book.location_id == station_id]
    # Compute buy/sell volume for each type
    for type_id in target_types:
        buy_volume, sell_volume = compute_side_volume(type_id, next_date, order_book, volume_thresh_map)
        side_volume_data.append(dict(day=next_date, type_id=type_id, buy_volume=buy_volume, sell_volume=sell_volume))
#
# Finally, we convert the side volume data into a dataframe
side_volume_df = pd.DataFrame(side_volume_data, index=[x['day'] for x in side_volume_data])

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [13]:
# We're now ready to apply our first two filters which require:
#
# 1. High buy side volume, we want mostly buyers in our market, which implies:
# 2. Low sell side volume, we want few sellers in our market.
#
# Without loss of generality, we'll impose a threshold on buy volume.  For this example, we'll use
# 80%.  That is, we'll keep a target type if trades against ask orders make up at least 80% of
# total volume.  It will likely be rare for this to happen every day in our date range, so we'll
# only require this threshold to be met for 75% of our target dates.
#
# Lowering the volume threshold, or lowering the required day count will admit more types,
# but we argue these will be less successful relisting targets.
#
sell_volume_threshold = 0.80
sell_day_count_threshold = 0.75
side_volume_df_copy = side_volume_df.copy()
side_volume_df_copy['total_volume'] = side_volume_df_copy.buy_volume + side_volume_df_copy.sell_volume
side_volume_df_copy['buy_ratio'] = side_volume_df_copy.buy_volume / side_volume_df_copy.total_volume
side_volume_df_copy['sell_ratio'] = side_volume_df_copy.sell_volume / side_volume_df_copy.total_volume
sell_exceeds_threshold = side_volume_df_copy[side_volume_df_copy.sell_ratio > sell_volume_threshold]
#
# We can now view for each asset type how many days meet our requirements
#
day_vol_counts = sell_exceeds_threshold.groupby(sell_exceeds_threshold.type_id).day.count()
print(day_vol_counts[day_vol_counts > len(sat_date_range) * sell_day_count_threshold])
#
# The index of this result represents our new target set
#
targets = day_vol_counts[day_vol_counts > len(sat_date_range) * sell_day_count_threshold].index
len(targets)

type_id
4405     17
11370    17
11578    17
21640    16
22291    16
26929    18
28668    18
31213    19
31358    16
31360    18
31372    17
31716    16
31788    19
31790    19
31796    19
40519    20
Name: day, dtype: int64


16

In [14]:
# Our next filter requires measuring competition on our types which we defined earlier as the number of
# existing orders which change price in a given time interval.  More competitors will make it more likely
# that someone will undercut our sell orders right after we buy out the ask side for relisting.
#
# We'll pull in our previous code for extracting order changes, except that we'll make a few small
# changes so that we only count sell order changes.  This is where we'll experience any competition.
#
def count_sell_order_changes(order_book, type_list, sample_interval, verbose=False):
    by_side = order_book[order_book.buy == False]
    samples = by_side.resample(sample_interval)
    total_samples = len(samples)
    changes = []
    if verbose:
        print("Checking %d samples for market participants" % total_samples, flush=True)    
    count = 0
    #
    for sample_group in samples:
        #
        # Each group is a pair (sample_time, sample_dataframe)
        sample_time = sample_group[0]
        sample = sample_group[1]
        if verbose:
            print("X", end='', flush=True)
            count += 1
            if count % 72 == 0:
                print()
        #
        # Iterate through each type in the type list
        for next_type in type_list:
            # Reduce this sample by type
            by_type = sample[sample.type_id == next_type]
            # Group by orders
            orders = by_type.groupby(['order_id'])
            # Count the unique prices for each order, flag those orders with more than
            # one price in the samnple interval.
            changed = orders['price'].nunique() > 1
            # Count how many orders changed price at least once in the sample interval.
            count = changed[changed == True].count()
            # Save the number of orders which changed prices
            changes.append(dict(time=sample_time, type_id=next_type, change_count=count))
    if verbose:
        print(flush=True)
    return pd.DataFrame(changes, index=[x['time'] for x in changes])

In [15]:
# Now we'll loop through every day in our date range and extract competition information.
# We'll use 30 minute intervals for his filter.  You can use a smaller interval if you plan
# to monitor your orders more frequently.
#
change_count_data = []
for next_date in sat_date_range:
    order_book = OrderBook.get_data_frame(dates=[next_date], types=targets, regions=[region_id], 
                                          config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                      fill_gaps=True, verbose=True))
    order_book = order_book[order_book.location_id == station_id]
    # Compute and store change count for this day
    change_count_data.append(count_sell_order_changes(order_book, targets, '30min'))

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [16]:
# Now that we have competition data, we need to determine what we'll consider as an 
# acceptable level of competition.  To help decide, we'll present four measures of
# competition as we've done in previous examples:
#
# 1. Average change count
# 2. Median change count
# 3. Max change count
# 4. Some other quantile of change count
#
# We'll also apply an optimization we applied before, which is to assume we'll only be monitoring
# our orders between 1200 UTC to 2400 UTC.
#
all_changes = change_count_data[0].append(change_count_data[1:])
constrained = all_changes[all_changes.index.hour >= 12]
pd.DataFrame({ 'Average': constrained.groupby(constrained.type_id).change_count.mean(),
               'Median' : constrained.groupby(constrained.type_id).change_count.median(),
               'Max': constrained.groupby(constrained.type_id).change_count.max(),
               '95%': constrained.groupby(constrained.type_id).change_count.quantile(0.95)})

,95%,Average,Max,Median
type_id,,,,
4405,4.00,1.129167,7,1
11370,2.00,0.397917,5,0
11578,2.00,0.397917,3,0
21640,3.00,0.622917,5,0
22291,3.00,0.712500,8,0
26929,3.00,0.477083,7,0
28668,3.00,0.581250,5,0
31213,2.00,0.406250,5,0
31358,4.00,1.147917,9,0


In [17]:
# What is acceptable competition?  This is highly subjective but obviously increasing your
# threshold will admit more types.  On average, all types look acceptable but the maximum
# values show areas of higher competition.  As in previous examples, we like using the 95%
# quantile as it helps reveal whether the maximum is a true outlier, or closer to actual behavior.
#
# For this example, we'll set our 95% quantile threshold at 4 meaning we'll accept any type
# with a 95% quantile order change count less than four.  Let's see what types this threshold
# will allow.
#
less_threshold = 4
less_count = constrained.groupby(constrained.type_id).change_count.quantile(0.95) < less_threshold
for x in less_count[less_count == True].index:
    print(str(x) + " " + market_type_map[x]['typeName'])
low_comp_targets = list(less_count[less_count == True].index)

11370 Prototype Cloaking Device I
11578 Covert Ops Cloaking Device II
21640 Valkyrie II
22291 Ballistic Control System II
26929 Small Processor Overclocking Unit I
28668 Nanite Repair Paste
31213 Small Gravity Capacitor Upgrade I
31716 Small Anti-EM Screen Reinforcer I


In [18]:
# From these last eight types, we're now ready to apply our last filter which will compute
# the buyout cost we need to incur in order to relist and profit at a given return
# target.
#
# This filter computes all transaction costs, so we'll define tax rate and broker rate here.
#
tax_rate = 0.01
broker_rate = 0.025
#
# Target return is the return on cost we intend to realize from relisting.  Lowering this
# value we reduce buyout cost, but of course also reduces profit.
#
target_return = 0.05

In [19]:
# To compute buyout, we'll accumulate buyout orders in each snapshot until we've accumulated enough
# orders so that we can sell at a price high enough to meet our return target.  We also require
# that there are no sell orders below our sell price in the order book.  Such orders would sell
# before us, and likely require that we reprice (which would not meet our return target).
#
# The function below computes the buyout amount for a single type on an order book representing
# a snapshot.
#
def compute_buyout(type_id, target_return, order_book, tax_rate, broker_rate):
    # Compute proceeds from sale less tax and broker fees
    def compute_net(amount, price):
        return (amount * price) - (amount * price * tax_rate) - (amount * price * broker_rate)
    # Compute minimum sell price based on current cost
    def compute_sell_price(amount, cost):
        if amount == 0:
            return None
        return (cost * (target_return + 1))/(amount * (1 - tax_rate - broker_rate))
    # Check whether there are orders below our required sell price.
    def check_target_met(order_list, cost, count):
        if len(order_list) == 0:
            return True
        price = compute_sell_price(count, cost)
        if price is None:
            return False
        return order_list[0]['price'] >= price
    # Now work through the book buying up the buy side until we can sell at a price
    # which meets our return target.
    asset_count = 0
    asset_cost = 0                                                
    by_type = order_book[order_book.type_id == type_id]
    by_side = by_type[by_type.buy == False]
    # Copy orders into consumable objects
    order_list = []
    for next_order in by_side.iterrows():
        order_list.append(dict(price=next_order[1]['price'], volume=next_order[1]['volume']))
    # Now consume until we meet our return target
    while not check_target_met(order_list, asset_cost, asset_count):
        asset_count += order_list[0]['volume']
        asset_cost += order_list[0]['price'] * order_list[0]['volume']
        order_list.pop(0)
    sell_price = compute_sell_price(asset_count, asset_cost)
    gross_gain = -1 if sell_price is None else compute_net(asset_count, sell_price)
    net_gain = -1 if gross_gain == -1 else gross_gain - asset_cost
    return (asset_count, asset_cost, sell_price, gross_gain, net_gain)    

In [20]:
# The following function computes buyout amounts from each snapshot for each type
# across the order book for a single day.  The result is returned as a DataFrame.
#
def collect_all_buyouts(order_book, targets, tax_rate, broker_rate, target_return):
    #
    # Iterate over book snapshots
    #
    buyout_results = []
    snap_list = list(order_book.groupby(order_book.index))
    total = len(targets)
    print("[" + ('-'*total) + "]")
    print("[", end='')
    for next_target in targets:
        print('+', end='')
        for next_group in snap_list:
            timestamp = next_group[0]
            snapshot = next_group[1]
            buyout_data = compute_buyout(next_target, target_return, snapshot, tax_rate, broker_rate)
            buyout_results.append(dict(time=timestamp, type_id=next_target, count=buyout_data[0], 
                                       price=buyout_data[2], cost=buyout_data[1], profit=buyout_data[4]))
    print(']')
    return pd.DataFrame(buyout_results, index=[x['time'] for x in buyout_results])

In [21]:
# Now we'll collect buyout data across all snapshots and all dates in our date
# range.  Because we are down to only a few types, we can request the entire order book
# at once.
#
order_book = OrderBook.get_data_frame(dates=sat_date_range, types=low_comp_targets, regions=[region_id], 
                                      config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                  fill_gaps=True, verbose=True))
order_book = order_book[order_book.location_id == station_id]
buyout_data = collect_all_buyouts(order_book, low_comp_targets, tax_rate, broker_rate, target_return)

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done
[--------]
[++++++++]


In [22]:
# The output data frame has the following columms:
#
# cost - the cost of buying out enough of the sell orders for each snapshot
# count - the number of units purchased to buy out a snapshot
# price - the price for the sell order we would need to place with our purchased assets to meet our return target
# profit - the profit we would expect to make if we sold all assets less sales tax and broker fees
#
# We've set our return target to guarantee we make at least 5% on our investment.  Let's look first
# at how this translates into profit.  We'll display the average profit across all snapshots for each
# type.
#
isk_format = lambda x: "{0:,.2f}".format(x)
buyout_data.groupby(buyout_data.type_id)['profit'].mean().sort_values().apply(isk_format)

type_id
31213       10,963,416.79
31716       11,429,221.53
26929       15,021,073.09
22291      375,556,540.29
11370      501,940,916.74
21640      537,484,436.48
11578      607,184,377.00
28668    1,598,871,006.66
Name: profit, dtype: object

In [23]:
# Some types report very high profit, but since we know these values represent a 5% return, 
# the corresponding cost of these investments must be large.
#
buyout_data.groupby(buyout_data.type_id)['cost'].mean().sort_values().apply(isk_format)

type_id
31213       219,268,335.85
31716       228,584,430.60
26929       300,421,461.89
22291     7,511,130,805.74
11370    10,038,818,334.73
21640    10,749,688,729.58
11578    12,143,687,540.09
28668    31,977,420,133.13
Name: cost, dtype: object

In [24]:
# Before we decide whether some or all of these opportunities are worth our time, it's important
# to look at the average unit count as compared to daily volume.  The average unit count represents
# the number of units we need to purchase to buy out a side, and therefore the number of units
# we'll need to sell after relisting.  Here are the results for this example:
#
buyout_data.groupby(buyout_data.type_id)['count'].mean().sort_values().apply(isk_format)

type_id
11578        2,376.15
26929        3,269.02
31213        3,572.03
31716        4,485.74
11370        6,101.84
22291        7,702.98
21640       10,609.14
28668    1,077,253.43
Name: count, dtype: object

In [25]:
# You may recall from above that we've already computed sell volume (that is, volume of trades buying
# from sell orders).  We can extract and average this data to get an idea how our counts compare to
# average sell volume.
#
side_volume_df_copy.groupby(side_volume_df_copy.type_id).sell_volume.mean()[buyout_data.type_id.unique()]

type_id
11370      1215.05
11578      1284.55
21640      2473.30
22291      3659.60
26929       997.75
28668    529547.20
31213      1043.15
31716      1489.05
Name: sell_volume, dtype: float64

In [26]:
# If we look at type 11578 as an example, we see that we'll need to sell about twice the average daily
# volume in order to sell all relisted assets.  Most of the other assets have a similar relationship
# (or worse).
#
# This analysis suggests that perhaps 5% is too aggressive of a return target.  Let's lower our target to
# 2% to see if there is any improvement.
#
order_book = OrderBook.get_data_frame(dates=sat_date_range, types=low_comp_targets, regions=[region_id], 
                                      config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                  fill_gaps=True, verbose=True))
order_book = order_book[order_book.location_id == station_id]
tp_buyout_data = collect_all_buyouts(order_book, low_comp_targets, tax_rate, broker_rate, 0.02)

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done
[--------]
[++++++++]


In [27]:
# Cost and count should both be lower, which is the case as shown below.
#
print(tp_buyout_data.groupby(buyout_data.type_id)['cost'].mean().sort_values().apply(isk_format))
print(tp_buyout_data.groupby(buyout_data.type_id)['count'].mean().sort_values().apply(isk_format))

type_id
31213       173,392,965.48
31716       224,912,762.15
26929       278,003,279.98
22291     6,620,699,134.93
11578     9,365,789,668.19
11370     9,760,760,562.63
21640    10,178,280,648.71
28668    27,288,382,029.60
Name: cost, dtype: object
type_id
11578      1,856.33
31213      2,901.74
26929      3,035.23
31716      4,434.19
11370      5,956.31
22291      6,840.73
21640     10,076.57
28668    924,684.12
Name: count, dtype: object


In [28]:
# If we compare counts to average sell volume again, we see that some types are a little more reasonable
# but it's still the case that every time will have to have an above average day.  This implies that
# any relisting strategy would need to extend into the next day.
#
side_volume_df_copy.groupby(side_volume_df_copy.type_id).sell_volume.mean()[tp_buyout_data.type_id.unique()]

type_id
11370      1215.05
11578      1284.55
21640      2473.30
22291      3659.60
26929       997.75
28668    529547.20
31213      1043.15
31716      1489.05
Name: sell_volume, dtype: float64

In [29]:
# Finally, for completeness, let's look at the profit for this case.
#
tp_buyout_data.groupby(buyout_data.type_id)['profit'].mean().sort_values().apply(isk_format)

type_id
31213      3,467,859.31
31716      4,498,255.24
26929      5,560,065.60
22291    132,413,982.70
11578    187,315,793.36
11370    195,215,211.25
21640    203,565,612.97
28668    545,767,640.59
Name: profit, dtype: object

The cost benefit for the relisting strategy doesn't seem very promising, at least for the region and thresholds we provided.  It is possible that this strategy works better in other environments with less overall competition.  We leave this analysis to the reader.